### Imports etc

In [1]:
import sys
from pathlib import Path
import pandas as pd
import seaborn as sns

REPO_PATH = Path.home() / "workspace/jetpointnet"
SCRIPT_PATH = REPO_PATH / "python_scripts"
sys.path.append(str(SCRIPT_PATH))

import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ''
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# from mpl_toolkits.axes_grid1 import make_axes_locatable
from jets_training.models.JetPointNet import PointNetSegmentation
from jets_training.jets_visualize import generate_images_and_metadata
from jets_training.jets_train import (
    TRAIN_INPUTS,
    MAX_SAMPLE_LENGTH,
    baseline_configuration,
    EXPERIMENT_NAME,
)
OUTPUT_ACTIVATION_FUNCTION = baseline_configuration['OUTPUT_ACTIVATION_FUNCTION']




2024-07-11 17:40:10.648590: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-11 17:40:10.699355: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 17:40:10.699389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 17:40:10.700896: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-11 17:40:10.709129: I tensorflow/core/platform/cpu_feature_guar

Logged in as jhimmens


# Generate Event Images

In [2]:
model_path = "/home/jhimmens/workspace/jetpointnet/models/rev_2/collected_data/PointNet_epoch=80_name=lucky-valley-926.keras"
sets_to_visualize = ["JZ4"]
max_file_per_set = 3
max_images_per_set = 10

MODEL_NAME = model_path.split('/')[-1]
VISUAL_PATH = REPO_PATH / "visualizations" / EXPERIMENT_NAME / MODEL_NAME

model = PointNetSegmentation(MAX_SAMPLE_LENGTH, 
                             num_features=len(TRAIN_INPUTS), 
                             num_classes=1, 
                             output_activation_function=OUTPUT_ACTIVATION_FUNCTION,
                             model_version=1)

model.load_weights(model_path)

generate_images_and_metadata(sets_to_visualize, model, max_file_per_set, max_images_per_set, VISUAL_PATH)

2024-07-11 17:40:14.449581: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-11 17:40:14.449612: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: triumf-ml2
2024-07-11 17:40:14.449618: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: triumf-ml2
2024-07-11 17:40:14.449724: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: INVALID_ARGUMENT: expected %d.%d, %d.%d.%d, or %d.%d.%d.%d form for driver version; got "1"
2024-07-11 17:40:14.449751: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 550.67.0


  0%|          | 0/1 [00:00<?, ?it/s]

JZ4: 0it [00:00, ?it/s]

/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_2/SavedNpz/deltaR=0.2_maxLen=800_EScale=1/val/JZ4/user.mswiatlo.39955735._000005.mltree.root_chunk_0_val.parquet.npz
 14/224 [>.............................] - ETA: 19s

KeyboardInterrupt: 